# Breadth-First Traversals
###### (https://doisinkidney.com/posts/2018-06-03-breadth-first-traversals-in-too-much-detail.html)

In [7]:
data Tree a = Node { rootLabel :: a, subForest :: [Tree a] }

tree :: Tree Int
tree = Node 1 [Node 2 [ Node 3 []
                      , Node 4 []
                      ]
              , Node 5 [Node 6 []]
              ]

We want to write a traversal in the lensy sense, so our solution should be of type

In [8]:
:ext RankNTypes
type BFT a b = forall f. Applicative f => (a -> f b) -> Tree a -> f (Tree b)

### Breadth-First Enumeration
Let's first look at making a list of elements in bread-first order

In [4]:
breadthFirstEnumerate :: Tree a -> [a]
breadthFirstEnumerate ts = f ts b []
  where
    f (Node x xs) fw bw = x : fw (xs : bw)
    b [] = []
    b qs = foldl (foldr f) b qs []
    
breadthFirstEnumerate tree

[1,2,5,3,4,6]

This works by popping items off the front of a queue and pushing the sub-forest on to the back of the queue. `fw` is the recursive function into which we pass the queue. It looks like we are adding to the front of the queue, but this not the case because it is being consumed in reverse with `foldl`.

Let's examine how the two nested folds work:
- the `foldl` works chunk-wise through the queue (a chunk being the subforest from a node)
- it will eventually produce a function that takes a list, to which we are passing the `[]`
- by using `b` as the acc argument, we build up the following composition:
  - `foldr f (foldr f (foldr f b chunk1) chunk2) chunk3` etc...
- the `foldr` steps through each individual item in the chunk, taking the queue as an argument, appending to it, then passing it to the next inner `foldr`.
- because of how `foldl` works, the last chunk in the queue will be operated on by the outer most `foldr`
- the outermost `foldr` operation gets passed an empty list (the queue) and does the following:
  - appends the root element of the node to the list we are building
  - the list is build by appending the subforest to the queue provided as an argument to the next step of the `foldr`.
  - for `foldr` step that operates on the last item in the last chunk, the queue is passed to a recursive call to `b`
  - otherwise, the queue is either passed to the next step in the `foldr`, or if it's the last item in the chunk, to the next step of the `foldl`, which will be an inner `foldr`.
- `b` consumes the queue it is passed whilst building a new one to be eventually passed to a recursive call.
- when `b` is finally passed an empty queue, the base case is reached, recursion ceases.

### Level-First Enumeration
As a preamble to the traversal, let's add some structure to the enumeration so that each level is grouped in it's own list.

Here is the (implicit) queue version, similiar to the previous enumeration.

In [9]:
levels :: Tree a -> [[a]]
levels ts = f b ts [] []
  where
    f k (Node x xs) ls qs = k (x : ls) (xs : qs)
    b _ [] = []
    b k qs = k : foldl (foldl f) b qs [] []
    
levels tree

[[1],[2,5],[3,4,6]]

let's see what the action of the nested `foldl`s looks like
- foldl f (foldl f (foldl f b chunk1) chunk2) chunk3
By having the inner fold be a left instead of right, the items in each chunk are processed from last to first so they are appended to the list in the correct order.

Note that if the list we are appending to was the acculumator of the left fold, the items would get reversed, but since the list we are appending to is an argument that will be passed to the iteration of the fold dealing with the last item, the items will be appended in the correct order as the list is propogated up through the fold, the first item in the chunk being the last one to get appended to the level list.

#### Zippy Style
An observation to be made here is that the implicit queue is just using a church encoded tuple to reverse the order of the fold. In otherwords, the result of the fold is a function to which we are passing the two lists. Let's instead pass the tuple of lists in directly and run it in one direction. Now the accumulator of the nested folds is the tuple rather than a recursive function

In [13]:
levels :: Tree a -> [[a]]
levels ts = b (f ts ([], []))
  where
    f (Node x xs) (ls, qs) = (x : ls, xs : qs)
    b (_, []) = []
    b (k, qs) = k : b (foldr (flip (foldr f)) ([], []) qs)
    
levels tree

[[1],[2,5],[3,4,6]]

We are only folding on the second element of pair, let's fold on this list directly instead of building the intermediate list. In fact, we're running a fold over the whole thing, which we can do straight away.

In [22]:
levels :: Show a => Tree a -> [[a]]
levels ts = f ts []
  where
    f (Node x xs) (q:qs) = (x : q) : foldr f qs xs
    f (Node x xs) []     = [x]     : foldr f [] xs
    
levels tree

[[1],[2,5],[3,4,6]]

here the resulting list is being built in such a manner that each level starts with the singleton list of the root element of the last node of that level (because of `foldr`) - and that call to `f` is going to produce the queue that gets passed to the last iteration of an outer `foldr`. So the queue is a list of chunks where each chunk is the list of items belonging to each level. At each node, we get the current queue, we pick off the current level from the queue, put the root node on it, then fold over the sub-trees with the tail of the queue as the accumulator so that the roots of the sub-tree nodes will be appended to the list for the next level.

Note that because `:` is lazy in it's second argument, each level is created without descending into the next level.

### Cofree
All the function defined so far can be generalized to work with Cofree. Note that the rose tree is recovered by using `[]` as the first type argument to Cofree.

In [8]:
data Cofree f a = a :< f (Cofree f a)

cofreeTree = 1 :< [2 :< [3 :< [], 4 :< []], 5 :< [6 :< []]]

### Traversal
We know how to visit each element in depth first order, so we can apply the applicative effect to each element. What we still need to figure out is how to reconstruct the tree afterwards.

The first thing we'll do is use a trick with `Traversable` where we can fill a container with the contents of a list.
```
fill [(),(),(),()] [1..] = ([1,2,3,4], [5..])
```
the result is function is the result of a state computation, first element is the filled container, second is the remaining state.

The key to this is a `pop` function that replaces each item in the traversable with a state action that pops off the element from the list state.

In [17]:
import Control.Monad.State

pop :: State [a] a
pop = state (\(x:xs) -> (x, xs))

We can then traverse this function over the container

In [22]:
fill :: Traversable t => t a -> State [b] (t b)
fill = traverse (const pop)

### Composing Applicatives
Applicatives compose nicely in a variety of ways. Given two applicatives, f and g, we can perform their effects on a list by either layering the effects (using `Compose`) or by running one after the other.

In our case we need to compose two applicatives, the one the user wants to run and our internal state. Instead of using `Compose`, we can define our own functions to avoid some additional calls to `pure`.

In [7]:
:m Control.Applicative

map2 :: (Functor f, Functor g) => (a -> b -> c) -> f a -> g b -> f (g c)
map2 f fa gb = fmap (\a -> fmap (\b -> f a b) gb) fa

app2 :: (Applicative f, Applicative g) => (a -> b -> c -> d) -> f a -> g b -> f (g c) -> f (g d)
app2 f fa gb = liftA2 (\a -> liftA2 (f a) gb) fa

The outer applicative `f` will be the user's effect, the `g` will be `State`
### Take 1: Zippy-style Traversing
First we'll try to convert the zippy style implementation to a traversal.

Start by using `Cofree` instead of `[]`

In [10]:
levels :: Show a => Cofree [] a -> [[a]]
levels ts = f ts []
  where
    f (x :< xs) (q:qs) = (x : q) : foldr f qs xs
    f (x :< xs) []     = [x]     : foldr f [] xs
    
levels cofreeTree

[[1],[2,5],[3,4,6]]

Now pair the root labels with the subforest

In [14]:
breadthFirst :: Cofree [] a -> [[(a, [Cofree [] a])]]
breadthFirst ts = f ts []
  where
    f (x :< xs) (q:qs) = ((x, xs) : q) : foldr f qs xs
    f (x :< xs) []     = [(x, xs)]     : foldr f [] xs

Next we want to call `fill` on the subforests

In [24]:
breadthFirst :: Cofree [] a -> [[(a, State [b] [b])]]
breadthFirst ts = f ts []
  where
    f (x :< xs) (q:qs) = ((x, fill xs) : q) : foldr f qs xs
    f (x :< xs) []     = [(x, fill xs)]     : foldr f [] xs

We can run the applicative effect on the root label

In [27]:
breadthFirst :: (a -> f a) -> Cofree [] a -> [[(f a, State [b] [b])]]
breadthFirst c ts = f ts []
  where
    f (x :< xs) (q:qs) = ((c x, fill xs) : q) : foldr f qs xs
    f (x :< xs) []     = [(c x, fill xs)]     : foldr f [] xs

Now we use the combinators we defined before to combine the two effects

In [32]:
breadthFirst :: Applicative f => (a -> f b) -> Cofree [] a -> [f (State [Cofree [] b] [Cofree [] b])]
breadthFirst c ts = f ts []
  where
    f (x :< xs) (q:qs) =
      app2 (\y ys zs -> (y :< ys) : zs) (c x) (fill xs) q : foldr f qs xs
    f (x :< xs) [] =
      map2 (\y ys -> [y :< ys]) (c x) (fill xs) : foldr f [] xs

This builds a list containing all the level-wise traversals of the tree. To combine them into one, we can use a fold.

In [42]:
breadthFirst :: (Traversable t, Applicative f) => (a -> f b) -> Cofree t a -> f (Cofree t b)
breadthFirst c ts =
  head <$> foldr (liftA2 evalState) (pure []) (f ts [])
  where
    f (x :< xs) (q:qs) =
      app2 (\y ys zs -> (y :< ys) : zs) (c x) (fill xs) q : foldr f qs xs
    f (x :< xs) [] =
      map2 (\y ys -> [y :< ys]) (c x) (fill xs) : foldr f [] xs
      
breadthFirst print cofreeTree

1
2
5
3
4
6

### Take 2: Queue-based Traversing
Now we convert the queue based enumeration to a traversal

In [47]:
--levels :: Tree a -> [[a]]
--levels ts = f b ts [] []
--  where
--    f k (Node x xs) ls qs = k (x : ls) (xs : qs)
--    b _ [] = []
--    b l qs = l : foldl (foldl f) b qs [] []
    
breadthFirst :: (Traversable t, Applicative f) => (a -> f b) -> Cofree t a -> f (Cofree t b)
breadthFirst c ts = head <$> f b ts e []
  where
    f k (x :< xs) ls qs =
      k (app2 (\y ys zs -> (y :< ys) : zs) (c x) (fill xs) ls) (xs : qs)
      
    b _ [] = pure []
    b l qs =
      liftA2 evalState l (foldl (foldl f) b qs e [])
    
    e = pure $ pure []